In [243]:
import pandas as pd

In [244]:
df = pd.DataFrame.from_csv('lending-club-data.csv')

D:\Users\zeyang\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [245]:
df['safe_loans'] = df['bad_loans'].apply(lambda x: 1 if x == 0 else -1)

In [246]:
del df['bad_loans']

In [247]:
len(df[df['safe_loans'] == 1]) / len(df)

0.8111853319957262

In [248]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

In [249]:
target = 'safe_loans'

In [250]:
df = df[features + [target]]

In [251]:
train_idx = pd.read_json('module-5-assignment-1-train-idx.json')[0]
validation_idx = pd.read_json('module-5-assignment-1-validation-idx.json')[0]

In [252]:
train_data = df.iloc[train_idx]
validation_data = df.iloc[validation_idx]

In [253]:
import sklearn
import sklearn.tree as tree
import numpy as np

In [254]:
train_data = pd.get_dummies(train_data)
validation_data = pd.get_dummies(validation_data)

In [269]:
new_features = train_data.columns.values.tolist()

In [271]:
new_features.remove('safe_loans')

In [273]:
train_features = train_data[new_features].as_matrix()
train_label = train_data[target].as_matrix()
validation_features = validation_data[new_features].as_matrix()
validation_label = validation_data[target].as_matrix()

In [308]:
# Depth = 6
decision_tree_model = tree.DecisionTreeClassifier(max_depth=6)

In [309]:
decision_tree_model.fit(train_features, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [310]:
small_model = tree.DecisionTreeClassifier(max_depth=2)

In [311]:
small_model.fit(train_features, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [312]:
# validation data set
validation_safe_loans = validation_data[validation_data[target] == 1][new_features].as_matrix()
validation_riskly_loans = validation_data[validation_data[target] == -1][new_features].as_matrix()

In [313]:
sample_validation_data_risky = validation_riskly_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

In [281]:
sample_validation_data = np.concatenate((sample_validation_data_safe, sample_validation_data_risky))

In [314]:
decision_tree_model.predict(sample_validation_data)

array([ 1, -1, -1,  1], dtype=int64)

In [237]:
# What percentage of the predictions on sample_validation_data did decision_tree_model get correct?
# 50%

In [315]:
decision_tree_model.predict_proba(sample_validation_data)

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [125]:
# Quiz Question: Which loan has the highest probability of being classified as a safe loan?
# 4th

In [316]:
predictions_probs = (decision_tree_model.predict_proba(validation_features)[:,1] >= 0.5).astype(int)

In [317]:
np.place(predictions_probs, predictions == 0, -1)

In [318]:
predictions = decision_tree_model.predict(validation_features)

In [319]:
# Checkpoint: Can you verify that for all the predictions with probability >= 0.5, the model predicted the label +1?
all(predictions[np.where(predictions_probs == 1)])

True

In [320]:
decision_tree_model.predict_proba(sample_validation_data)

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [321]:
small_model.predict(sample_validation_data)

array([ 1, -1, -1,  1], dtype=int64)

In [322]:
small_model.predict_proba(sample_validation_data)

array([[ 0.41896585,  0.58103415],
       [ 0.59255339,  0.40744661],
       [ 0.59255339,  0.40744661],
       [ 0.23120112,  0.76879888]])

In [323]:
# 2nd and 3rd have the same probability which means they fall into the same leaf

In [292]:
# Evaluate the accuracy

In [293]:
small_model.score(train_features, train_label)

0.61350204169353106

In [294]:
decision_tree_model.score(train_features, train_label)

0.64052761659144641

In [295]:
small_model.score(validation_features, validation_label)

0.61934510986643687

In [296]:
decision_tree_model.score(validation_features, validation_label)

0.63614821197759586

In [298]:
# Quiz Question: What is the accuracy of decision_tree_model on the validation set, rounded to the nearest .01?
# 0.64

In [299]:
# Complex model

In [300]:
big_model = tree.DecisionTreeClassifier(max_depth=10)

In [301]:
big_model.fit(train_features, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [302]:
big_model.score(train_features, train_label)

0.66379217709004945

In [303]:
big_model.score(validation_features, validation_label)

0.62677725118483407

In [304]:
# Quiz Question: How does the performance of big_model on the validation set compare to decision_tree_model 
# on the validation set? Is this a sign of overfitting?

# big_model's accuracy is 0.63 while tree_1's accuracy is 0.63
# This is a sign of overfitting, as model complexity is increasing while performace is not increasing

In [325]:
def cost(model, features, label, false_negative_cost, false_positive_cost):
    predictions = model.predict(features)
    errors = np.where(predictions != label)
    num_false_negative = len(np.where(label[errors] == 1)[0])
    num_false_positive = len(np.where(label[errors] == -1)[0])
    return false_negative_cost * num_false_negative + false_positive_cost * num_false_positive

In [328]:
cost(decision_tree_model, validation_features, validation_label, 10000, 20000)

50390000